In [1]:
from __future__ import print_function, division
from data_utils import load_CIFAR10

X_train, y_train, X_test, y_test = load_CIFAR10("cifar-10-batches-py")

In [3]:
import numpy as np
from skimage.feature import hog
from skimage import color, exposure

orie = 9
ppc = (8,8)
cpb = (2,2)
tmp_train = []
for i in xrange(X_train.shape[0]):
    gray_img = color.rgb2gray(X_train[i])
    fd = hog(gray_img, orientations=orie, pixels_per_cell=ppc,
                    cells_per_block=cpb, visualise=False, transform_sqrt=True)
    fd = fd[np.newaxis, :]
    tmp_train.append(fd)
hog_train = np.concatenate(tmp_train, axis=0)

tmp_test = []
for i in xrange(X_test.shape[0]):
    gray_img = color.rgb2gray(X_test[i])
    fd = hog(gray_img, orientations=orie, pixels_per_cell=ppc,
                    cells_per_block=cpb, visualise=False, transform_sqrt=True)
    fd = fd[np.newaxis, :]
    tmp_test.append(fd)

hog_test= np.concatenate(tmp_test, axis=0)

print(hog_train.shape)
print(hog_test.shape)

(50000, 324)
(10000, 324)


In [5]:
from skimage.feature import local_binary_pattern

radius = 3
n_points = 8 * radius
METHOD = 'uniform'
eps = 1e-7

lbp_train = []
for i in xrange(X_train.shape[0]):
    gray_img = color.rgb2gray(X_train[i])
    lbp = local_binary_pattern(gray_img, n_points, radius, METHOD)
    h, _ = np.histogram(lbp.ravel(),bins=np.arange(0, n_points+3), range=(0, n_points+2))
    h = h / (h.sum()+eps)
    h = h[np.newaxis, :]
    lbp_train.append(h)
    
lbp_train = np.concatenate(lbp_train, axis=0)

lbp_test = []
for i in xrange(X_test.shape[0]):
    gray_img = color.rgb2gray(X_test[i])
    lbp = local_binary_pattern(gray_img, n_points, radius, METHOD)
    h, _ = np.histogram(lbp.ravel(),bins=np.arange(0, n_points+3), range=(0, n_points+2))
    h = h / (h.sum()+eps)
    h = h[np.newaxis, :]
    lbp_test.append(h)

lbp_test = np.concatenate(lbp_test, axis=0)

print(lbp_train.shape)
print(lbp_test.shape)   

(50000, 26)
(10000, 26)


In [6]:
data_train = np.concatenate([hog_train, lbp_train], axis=1)
data_test = np.concatenate([hog_test, lbp_test], axis=1)
print(data_train.shape)
print(data_test.shape)

(50000, 350)
(10000, 350)


In [7]:
from sklearn import neighbors
import time 

group_K = [1, 3, 5, 20, 50, 100]
acc = np.zeros((len(group_K),))
for i in xrange(len(group_K)):
    print("K: %d" %(group_K[i]))
    clf = neighbors.KNeighborsClassifier(group_K[i], n_jobs=-1)
    clf.fit(data_train, y_train)
    tic = time.time()
    preds = clf.predict(data_test)
    toc = time.time()
    print("time for prediction: %d seconds" %(toc-tic))

    acc[i] = (preds==y_test).mean()
    print("accuracy: ", acc[i])

print(acc)

K: 1
time for prediction: 58 seconds
accuracy:  0.4606
K: 3
time for prediction: 64 seconds
accuracy:  0.4741
K: 5
time for prediction: 75 seconds
accuracy:  0.4897
K: 20
time for prediction: 50 seconds
accuracy:  0.4969
K: 50
time for prediction: 77 seconds
accuracy:  0.4772
K: 100
time for prediction: 74 seconds
accuracy:  0.4588
[ 0.4606  0.4741  0.4897  0.4969  0.4772  0.4588]


In [8]:
from sklearn import svm

tic = time.time()
lin_clf = svm.LinearSVC() 

lin_clf.fit(data_train, y_train)
toc_1 = time.time()
print("time for training: %d seconds" %(toc_1-tic))

preds = lin_clf.predict(data_test)
toc_2 = time.time()
print("time for prediction: %d seconds" %(toc_2-toc_1))

acc = (preds==y_test).mean()
print("accuracy: ", acc)

time for training: 24 seconds
time for prediction: 0 seconds
accuracy:  0.5507


In [9]:
from sklearn import linear_model

tic = time.time()
lin_reg = linear_model.LogisticRegression(multi_class='multinomial', n_jobs=-1, solver='sag', max_iter=500)
lin_reg.fit(data_train, y_train)
toc_1 = time.time()
print("time for training: %d seconds" %(toc_1-tic))

preds = lin_reg.predict(data_test)
toc_2 = time.time()
print("time for prediction: %d seconds" %(toc_2-toc_1))

acc = (preds==y_test).mean()
print("accuracy: ", acc)

time for training: 24 seconds
time for prediction: 0 seconds
accuracy:  0.5359
